In [1]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
from secrets import IEX_CLOUD_API_TOKEN

In [3]:
symbol = 'AAPL'
api_url = f'https://cloud.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'avgTotalVolume': 52277009,
 'calculationPrice': 'close',
 'change': -1.97,
 'changePercent': -0.0101,
 'close': None,
 'closeSource': 'official',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': None,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 193.17,
 'iexCloseTime': 1689883199531,
 'iexLastUpdated': 1689883199531,
 'iexMarketPercent': 20.209467901552863,
 'iexOpen': 195.095,
 'iexOpenTime': 1689859802642,
 'iexRealtimePrice': 193.17,
 'iexRealtimeSize': 200,
 'iexVolume': 1019022,
 'lastTradeTime': 1689883199995,
 'latestPrice': 193.13,
 'latestSource': 'Close',
 'latestTime': 'July 20, 2023',
 'latestUpdate': 1689883200128,
 'latestVolume': None,
 'low': None,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 30

In [4]:
pe_ratio = data['peRatio']
pe_ratio

32.85

In [5]:
 data['latestPrice']


193.13

In [6]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   
        
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [8]:
import pandas as pd
import requests

# Assuming 'my_columns' is defined

final_dataframe = pd.DataFrame(columns=my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        if symbol in data:
            try:
                latest_price = data[symbol]['quote']['latestPrice']
            except KeyError:
                latest_price = None  # Set to None if latest price is missing

            try:
                pe_ratio = data[symbol]['quote']['peRatio']
            except KeyError:
                pe_ratio = None  # Set to None if P/E ratio is missing

            final_dataframe = final_dataframe.append(
                pd.Series([symbol, latest_price, pe_ratio, 'N/A'], index=my_columns),
                ignore_index=True
            )
final_dataframe

# Fill missing values with the average of the respective columns
#final_dataframe['latestPrice'].fillna(final_dataframe['latestPrice'].mean(), inplace=True)
#final_dataframe['peRatio'].fillna(final_dataframe['peRatio'].mean(), inplace=True)

#print(final_dataframe)

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,124.55,27.43,N/A
1,AAL,17.44,6.03,N/A
2,AAP,70.01,10.4,N/A
3,AAPL,193.13,32.85,N/A
4,ABBV,142.19,33.54,N/A
...,...,...,...,...
496,YUM,135.88,31.9,N/A
497,ZBH,142.73,66.7,N/A
498,ZBRA,304.54,38.89,N/A
499,ZION,37.9,3.88,N/A


In [10]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0]
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)

In [11]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print("That's not a number! \n Try again:")
        portfolio_size = input("Enter the value of your portfolio:")

In [12]:
portfolio_input()

Enter the value of your portfolio:5426665595


In [13]:
portfolio_size

'5426665595'

In [14]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,FRC,3.51,0.43,30921171
1,DISH,7.25,2.21,14970111
2,VLO,120.73,3.42,898975
3,MPC,124.25,3.68,873507
4,ZION,37.9,3.88,2863675
5,PSX,104.05,4.01,1043088
6,SIVB,106.04,4.18,1023512
7,MOS,38.64,4.75,2808833
8,PFG,83.53,4.86,1299333
9,CF,79.02,5.25,1373491


In [16]:
symbol = 'AAPL'
batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch/?types=advanced-stats,quote&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(batch_api_call_url).json()

# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']

# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']

#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']

# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda

# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit

In [17]:
rv_columns = [
    'Ticker',
    'Price',
    'Number of Shares to Buy', 
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score'
]

rv_dataframe = pd.DataFrame(columns = rv_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://cloud.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,advanced-stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
            
        rv_dataframe = rv_dataframe.append(
            pd.Series([
                symbol,
                data[symbol]['quote']['latestPrice'],
                'N/A',
                data[symbol]['quote']['peRatio'],
                'N/A',
                data[symbol]['advanced-stats']['priceToBook'],
                'N/A',
                data[symbol]['advanced-stats']['priceToSales'],
                'N/A',
                ev_to_ebitda,
                'N/A',
                ev_to_gross_profit,
                'N/A',
                'N/A'
        ],
        index = rv_columns),
            ignore_index = True
        )

KeyError: 'DISCA'

In [18]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
6,ABMD,381.02,N/A,65.69,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
28,ALXN,182.5,N/A,59.45,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
40,AON,339.61,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
71,BRK.B,346.61,N/A,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
88,CERN,94.92,N/A,47.94,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
118,CTL,11,N/A,9.73,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A
121,CTXS,103.9,N/A,39.96,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A


In [23]:
for column in ['Price-to-Earnings Ratio', 'Price-to-Book Ratio','Price-to-Sales Ratio',  'EV/EBITDA','EV/GP']:
    rv_dataframe[column].fillna(rv_dataframe[column].mean(), inplace = True)

In [24]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score


In [21]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}

for row in rv_dataframe.index:
    for metric in metrics.keys():
        rv_dataframe.loc[row, metrics[metric]] = stats.percentileofscore(rv_dataframe[metric], rv_dataframe.loc[row, metric])/100

# Print each percentile score to make sure it was calculated properly
for metric in metrics.values():
    print(rv_dataframe[metric])

#Print the entire DataFrame    
rv_dataframe

0      0.592593
1      0.081481
2           0.2
3      0.688889
4      0.703704
         ...   
130    0.318519
131    0.444444
132    0.155556
133    0.540741
134    0.755556
Name: PE Percentile, Length: 135, dtype: object
0      0.740741
1      0.088889
2       0.22963
3       0.97037
4      0.925926
         ...   
130    0.725926
131    0.437037
132    0.340741
133    0.562963
134    0.237037
Name: PB Percentile, Length: 135, dtype: object
0      0.762963
1      0.022222
2      0.044444
3      0.918519
4      0.725926
         ...   
130    0.148148
131    0.340741
132    0.222222
133    0.807407
134    0.362963
Name: PS Percentile, Length: 135, dtype: object
0      0.740741
1      0.103704
2      0.133333
3      0.837037
4      0.311111
         ...   
130    0.259259
131    0.511111
132    0.140741
133    0.674074
134    0.518519
Name: EV/EBITDA Percentile, Length: 135, dtype: object
0      0.777778
1      0.037037
2      0.088889
3      0.977778
4       0.62963
         ...   
1

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.55,N/A,27.43,0.592593,6.36,0.740741,5.2300,0.762963,19.347658,0.740741,9.978417,0.777778,N/A
1,AAL,17.44,N/A,6.03,0.081481,-1.97,0.088889,0.2179,0.022222,6.057114,0.103704,0.670836,0.037037,N/A
2,AAP,70.01,N/A,10.40,0.2,1.58,0.22963,0.3716,0.044444,6.544879,0.133333,1.184627,0.088889,N/A
3,AAPL,193.13,N/A,32.85,0.688889,48.87,0.97037,7.8900,0.918519,24.973561,0.837037,18.590802,0.977778,N/A
4,ABBV,142.19,N/A,33.54,0.703704,18.90,0.925926,4.4200,0.725926,11.706692,0.311111,7.773852,0.62963,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,168.79,N/A,15.89,0.318519,6.24,0.725926,0.9632,0.148148,10.788785,0.259259,3.654360,0.244444,N/A
131,DGX,143.6,N/A,21.12,0.444444,2.66,0.437037,1.6700,0.340741,14.526312,0.511111,6.139710,0.407407,N/A
132,DHI,125.42,N/A,8.44,0.155556,2.07,0.340741,1.2700,0.222222,6.595140,0.140741,4.565392,0.288889,N/A
133,DHR,244.38,N/A,26.31,0.540741,3.62,0.562963,5.8300,0.807407,18.318197,0.674074,10.356759,0.792593,N/A


In [26]:
from statistics import mean

for row in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[row, metrics[metric]])
    rv_dataframe.loc[row, 'RV Score'] = mean(value_percentiles)
    
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,A,124.55,N/A,27.43,0.592593,6.36,0.740741,5.2300,0.762963,19.347658,0.740741,9.978417,0.777778,0.722963
1,AAL,17.44,N/A,6.03,0.081481,-1.97,0.088889,0.2179,0.022222,6.057114,0.103704,0.670836,0.037037,0.066667
2,AAP,70.01,N/A,10.40,0.2,1.58,0.22963,0.3716,0.044444,6.544879,0.133333,1.184627,0.088889,0.139259
3,AAPL,193.13,N/A,32.85,0.688889,48.87,0.97037,7.8900,0.918519,24.973561,0.837037,18.590802,0.977778,0.878519
4,ABBV,142.19,N/A,33.54,0.703704,18.90,0.925926,4.4200,0.725926,11.706692,0.311111,7.773852,0.62963,0.659259
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,DG,168.79,N/A,15.89,0.318519,6.24,0.725926,0.9632,0.148148,10.788785,0.259259,3.654360,0.244444,0.339259
131,DGX,143.6,N/A,21.12,0.444444,2.66,0.437037,1.6700,0.340741,14.526312,0.511111,6.139710,0.407407,0.428148
132,DHI,125.42,N/A,8.44,0.155556,2.07,0.340741,1.2700,0.222222,6.595140,0.140741,4.565392,0.288889,0.22963
133,DHR,244.38,N/A,26.31,0.540741,3.62,0.562963,5.8300,0.807407,18.318197,0.674074,10.356759,0.792593,0.675556


In [27]:
rv_dataframe.sort_values(by = 'RV Score', inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)

In [28]:
portfolio_input()

Enter the value of your portfolio:544785564555


In [29]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for i in range(0, len(rv_dataframe['Ticker'])-1):
    rv_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][i])
rv_dataframe


,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,AAL,17.44,624754087,6.03,0.081481,-1.97000,0.088889,0.217900,0.022222,6.057114,0.103704,0.670836,0.037037,0.066667
1,C,47.41,229818841,6.63,0.103704,0.49080,0.096296,0.784200,0.118519,3.832551,0.044444,0.829078,0.059259,0.084444
2,AIG,60.24,180871701,7.68,0.133333,1.02000,0.133333,0.844700,0.133333,3.311473,0.037037,0.844726,0.066667,0.100741
3,ALL,111.98,97300511,-12.48,0.037037,1.91000,0.311111,0.556800,0.081481,6.632302,0.148148,0.556822,0.014815,0.118519
4,COF,114.99,94753555,7.91,0.140741,0.80330,0.111111,1.060000,0.17037,5.832080,0.088889,1.317684,0.096296,0.121481
5,CFG,31.22,348997799,7.50,0.125926,0.68100,0.103704,1.470000,0.251852,4.445459,0.066667,1.572987,0.111111,0.131852
6,AAP,70.01,155630785,10.40,0.2,1.58000,0.22963,0.371600,0.044444,6.544879,0.133333,1.184627,0.088889,0.139259
7,CMA,52.93,205851337,5.59,0.074074,1.24000,0.155556,1.640000,0.325926,4.014566,0.051852,1.676518,0.125926,0.146667
8,BWA,46.14,236144588,11.31,0.222222,1.45000,0.185185,0.671400,0.103704,6.258760,0.118519,4.547238,0.274074,0.180741
9,DFS,102.45,106351501,6.91,0.111111,1.96000,0.322222,1.570000,0.296296,4.520725,0.074074,1.964123,0.155556,0.191852


In [30]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

In [31]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

float_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        })

In [32]:
column_formats = {
                    'A': ['Ticker', string_template],
                    'B': ['Price', dollar_template],
                    'C': ['Number of Shares to Buy', integer_template],
                    'D': ['Price-to-Earnings Ratio', float_template],
                    'E': ['PE Percentile', percent_template],
                    'F': ['Price-to-Book Ratio', float_template],
                    'G': ['PB Percentile',percent_template],
                    'H': ['Price-to-Sales Ratio', float_template],
                    'I': ['PS Percentile', percent_template],
                    'J': ['EV/EBITDA', float_template],
                    'K': ['EV/EBITDA Percentile', percent_template],
                    'L': ['EV/GP', float_template],
                    'M': ['EV/GP Percentile', percent_template],
                    'N': ['RV Score', percent_template]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

In [33]:
writer.save()